In [95]:
from torch import nn
import torch

import sys, os
import argparse
import datetime
import copy
import heapq

import sentencepiece as spm
import numpy as np

In [96]:
import requests
import rdkit
import pandas as pd

path='./Nat_Comm_2019/'
file=path+'SMRT_dataset.csv'

f=pd.read_csv(file,sep=';')
f

,pubchem,rt,inchi
0,5139,93.5,"InChI=1S/C3H8N2S/c1-2-6-3(4)5/h2H2,1H3,(H3,4,5)"
1,3505,687.8,InChI=1S/C19H25Cl2N3O3/c1-27-19(26)23-8-9-24(1...
2,2159,590.7,InChI=1S/C17H27N3O4S/c1-4-20-8-6-7-12(20)11-19...
3,1340,583.6,InChI=1S/C9H7NO2/c11-8-3-1-2-7-6(8)4-5-10-9(7)...
4,3344,579.0,InChI=1S/C15H20N2O2/c18-14-16-12-15(19-14)7-10...
...,...,...,...
80033,97733655,946.4,InChI=1S/C25H29N3O6S/c1-5-24(29)26-25-17(4)27-...
80034,98666786,653.1,InChI=1S/C17H24FN3O5S/c1-25-7-3-6-19-17(22)20-...
80035,98670835,648.2,InChI=1S/C17H25N3O5S/c1-13-4-3-5-16(8-13)26(22...
80036,98779314,783.9,InChI=1S/C21H25N3O4S/c1-15-7-9-19(10-8-15)29(2...


In [97]:
smilesTmp=[]
error_list=[]
for z,item in enumerate(inchi):
#     print(item)
    try:
        out = rdkit.Chem.inchi.MolFromInchi(item)
        out = rdkit.Chem.MolToSmiles(out)
        smilesTmp.append(out)
    except:
        error_list.append(z)
        smilesTmp.append('NaN')
    

smiles=pd.DataFrame(smilesTmp, columns=['smiles'])
df=pd.concat([f,smiles],axis=1)
df

RDKit ERROR: [18:44:47] ERROR: Explicit valence for atom # 16 N, 4, is greater than permitted
RDKit ERROR: [19:36:49] Explicit valence for atom # 14 N, 4, is greater than permitted
RDKit ERROR: [19:36:49] ERROR: Explicit valence for atom # 14 N, 4, is greater than permitted
RDKit ERROR: [19:36:49] Explicit valence for atom # 16 N, 4, is greater than permitted
[18:44:47] ERROR: Explicit valence for atom # 16 N, 4, is greater than permitted
[19:36:49] Explicit valence for atom # 14 N, 4, is greater than permitted
RDKit ERROR: [19:36:49] ERROR: Explicit valence for atom # 16 N, 4, is greater than permitted
RDKit ERROR: [19:36:49] Explicit valence for atom # 16 N, 4, is greater than permitted
[19:36:49] ERROR: Explicit valence for atom # 14 N, 4, is greater than permitted
[19:36:49] Explicit valence for atom # 16 N, 4, is greater than permitted
[19:36:49] ERROR: Explicit valence for atom # 16 N, 4, is greater than permitted
[19:36:49] Explicit valence for atom # 16 N, 4, is greater than pe

RDKit ERROR: [19:36:50] ERROR: Explicit valence for atom # 14 N, 4, is greater than permitted
RDKit ERROR: [19:36:50] Explicit valence for atom # 21 N, 4, is greater than permitted
[19:36:50] ERROR: Explicit valence for atom # 14 N, 4, is greater than permitted
[19:36:50] Explicit valence for atom # 21 N, 4, is greater than permitted
RDKit ERROR: [19:36:50] ERROR: Explicit valence for atom # 21 N, 4, is greater than permitted
RDKit ERROR: [19:36:50] Explicit valence for atom # 24 N, 4, is greater than permitted
[19:36:50] ERROR: Explicit valence for atom # 21 N, 4, is greater than permitted
[19:36:50] Explicit valence for atom # 24 N, 4, is greater than permitted
RDKit ERROR: [19:36:50] ERROR: Explicit valence for atom # 24 N, 4, is greater than permitted
RDKit ERROR: [19:36:50] Explicit valence for atom # 20 N, 4, is greater than permitted
[19:36:50] ERROR: Explicit valence for atom # 24 N, 4, is greater than permitted
[19:36:50] Explicit valence for atom # 20 N, 4, is greater than pe

RDKit ERROR: [19:36:53] ERROR: Explicit valence for atom # 19 N, 4, is greater than permitted
RDKit ERROR: [19:36:53] Explicit valence for atom # 15 N, 4, is greater than permitted
RDKit ERROR: [19:36:53] ERROR: Explicit valence for atom # 15 N, 4, is greater than permitted
RDKit ERROR: [19:36:53] Explicit valence for atom # 19 N, 4, is greater than permitted
[19:36:53] ERROR: Explicit valence for atom # 19 N, 4, is greater than permitted
[19:36:53] Explicit valence for atom # 15 N, 4, is greater than permitted
[19:36:53] ERROR: Explicit valence for atom # 15 N, 4, is greater than permitted
[19:36:53] Explicit valence for atom # 19 N, 4, is greater than permitted
RDKit ERROR: [19:36:53] ERROR: Explicit valence for atom # 19 N, 4, is greater than permitted
[19:36:53] ERROR: Explicit valence for atom # 19 N, 4, is greater than permitted
[19:36:53] Explicit valence for atom # 22 N, 4, is greater than permitted
RDKit ERROR: [19:36:53] Explicit valence for atom # 22 N, 4, is greater than pe

,pubchem,rt,inchi,smiles
0,5139,93.5,"InChI=1S/C3H8N2S/c1-2-6-3(4)5/h2H2,1H3,(H3,4,5)",CCSC(=N)N
1,3505,687.8,InChI=1S/C19H25Cl2N3O3/c1-27-19(26)23-8-9-24(1...,COC(=O)N1CCN(C(=O)Cc2ccc(Cl)c(Cl)c2)[C@H](CN2C...
2,2159,590.7,InChI=1S/C17H27N3O4S/c1-4-20-8-6-7-12(20)11-19...,CCN1CCC[C@@H]1CN=C(O)c1cc(S(=O)(=O)CC)c(N)cc1OC
3,1340,583.6,InChI=1S/C9H7NO2/c11-8-3-1-2-7-6(8)4-5-10-9(7)...,Oc1cccc2c(O)nccc12
4,3344,579.0,InChI=1S/C15H20N2O2/c18-14-16-12-15(19-14)7-10...,OC1=NCC2(CCN(CCc3ccccc3)CC2)O1
...,...,...,...,...
80033,97733655,946.4,InChI=1S/C25H29N3O6S/c1-5-24(29)26-25-17(4)27-...,CCOc1ccc(OCC)c(NS(=O)(=O)c2ccc(/C=C/c3onc(C)c3...
80034,98666786,653.1,InChI=1S/C17H24FN3O5S/c1-25-7-3-6-19-17(22)20-...,COCCCN=C(O)N1C[C@@H]2CN(S(=O)(=O)c3cccc(F)c3)C...
80035,98670835,648.2,InChI=1S/C17H25N3O5S/c1-13-4-3-5-16(8-13)26(22...,COCCN=C(O)N1C[C@@H]2CN(S(=O)(=O)c3cccc(C)c3)C[...
80036,98779314,783.9,InChI=1S/C21H25N3O4S/c1-15-7-9-19(10-8-15)29(2...,Cc1ccc(S(=O)(=O)N2C[C@@H]3CN(C(O)=Nc4ccccc4C)C...


In [98]:
df.drop(df[df['smiles'] == 'NaN'].index,inplace=True)
    
df

,pubchem,rt,inchi,smiles
0,5139,93.5,"InChI=1S/C3H8N2S/c1-2-6-3(4)5/h2H2,1H3,(H3,4,5)",CCSC(=N)N
1,3505,687.8,InChI=1S/C19H25Cl2N3O3/c1-27-19(26)23-8-9-24(1...,COC(=O)N1CCN(C(=O)Cc2ccc(Cl)c(Cl)c2)[C@H](CN2C...
2,2159,590.7,InChI=1S/C17H27N3O4S/c1-4-20-8-6-7-12(20)11-19...,CCN1CCC[C@@H]1CN=C(O)c1cc(S(=O)(=O)CC)c(N)cc1OC
3,1340,583.6,InChI=1S/C9H7NO2/c11-8-3-1-2-7-6(8)4-5-10-9(7)...,Oc1cccc2c(O)nccc12
4,3344,579.0,InChI=1S/C15H20N2O2/c18-14-16-12-15(19-14)7-10...,OC1=NCC2(CCN(CCc3ccccc3)CC2)O1
...,...,...,...,...
80033,97733655,946.4,InChI=1S/C25H29N3O6S/c1-5-24(29)26-25-17(4)27-...,CCOc1ccc(OCC)c(NS(=O)(=O)c2ccc(/C=C/c3onc(C)c3...
80034,98666786,653.1,InChI=1S/C17H24FN3O5S/c1-25-7-3-6-19-17(22)20-...,COCCCN=C(O)N1C[C@@H]2CN(S(=O)(=O)c3cccc(F)c3)C...
80035,98670835,648.2,InChI=1S/C17H25N3O5S/c1-13-4-3-5-16(8-13)26(22...,COCCN=C(O)N1C[C@@H]2CN(S(=O)(=O)c3cccc(C)c3)C[...
80036,98779314,783.9,InChI=1S/C21H25N3O4S/c1-15-7-9-19(10-8-15)29(2...,Cc1ccc(S(=O)(=O)N2C[C@@H]3CN(C(O)=Nc4ccccc4C)C...


In [102]:
import pickle

with open('./transformer_data.pkl', 'wb') as fi:
    pickle.dump(df, fi)
    
          

In [ ]:
print("length : ",len(smilesTmp))
cid=[]
df=f
for z in error_list:
    cid.append(f["pubchem"][z])  
print(cid,len(cid))
for z in cid:
    df.drop(df[df['pubchem'] == z].index,inplace=True)
    
df

In [67]:
inchi=df["inchi"]
smilesTmp=[]
error_list=[]
for z,item in enumerate(inchi):
#     print(item)
    try:
        out = rdkit.Chem.inchi.MolFromInchi(item)
        out = rdkit.Chem.MolToSmiles(out)
        smilesTmp.append(out)
    except:
        error_list.append(z)
        smilesTmp.append(item)
    
print("length : ",len(smilesTmp))

print(error_list)

length :  79957
[]


In [73]:
smiles=pd.DataFrame(smilesTmp, columns=["smiles"])
smiles

df_conv=pd.concat([df,smiles],axis=1)
df_conv

,pubchem,rt,inchi,smiles
0,5139.0,93.5,"InChI=1S/C3H8N2S/c1-2-6-3(4)5/h2H2,1H3,(H3,4,5)",CCSC(=N)N
1,3505.0,687.8,InChI=1S/C19H25Cl2N3O3/c1-27-19(26)23-8-9-24(1...,COC(=O)N1CCN(C(=O)Cc2ccc(Cl)c(Cl)c2)[C@H](CN2C...
2,2159.0,590.7,InChI=1S/C17H27N3O4S/c1-4-20-8-6-7-12(20)11-19...,CCN1CCC[C@@H]1CN=C(O)c1cc(S(=O)(=O)CC)c(N)cc1OC
3,1340.0,583.6,InChI=1S/C9H7NO2/c11-8-3-1-2-7-6(8)4-5-10-9(7)...,Oc1cccc2c(O)nccc12
4,3344.0,579.0,InChI=1S/C15H20N2O2/c18-14-16-12-15(19-14)7-10...,OC1=NCC2(CCN(CCc3ccccc3)CC2)O1
...,...,...,...,...
80033,97733655.0,946.4,InChI=1S/C25H29N3O6S/c1-5-24(29)26-25-17(4)27-...,NaN
80034,98666786.0,653.1,InChI=1S/C17H24FN3O5S/c1-25-7-3-6-19-17(22)20-...,NaN
80035,98670835.0,648.2,InChI=1S/C17H25N3O5S/c1-13-4-3-5-16(8-13)26(22...,NaN
80036,98779314.0,783.9,InChI=1S/C21H25N3O4S/c1-15-7-9-19(10-8-15)29(2...,NaN


In [77]:
print(df.shape)
print(smiles.shape)

(79957, 3)
(79957, 1)


In [58]:
cid=[]

df=f

for z in error_list:
    cid.append(f["pubchem"][z])
    
print(cid,len(cid))


for z in cid:
    df.drop(df[df['pubchem'] == z].index,inplace=True)
    
df

[298, 4477, 4022, 227516, 255871, 2777391, 2877960, 2919993, 2921419, 2948687, 2960737, 286826, 286876, 3123289, 309163, 3417144, 3463933, 3473868, 325419, 332278, 332429, 3565913, 3693695, 366651, 362112, 355940, 366721, 3854666, 3953303, 3979147, 4075240, 4176377, 4248743, 4427784, 4454455, 4532366, 4601040, 4647379, 28061, 5019999, 5064265, 5351231, 6002117, 6516747, 672387, 694700, 7205668, 711354, 7360729, 7518541, 7544267, 792698, 942434, 1040939, 1049537, 1071978, 1116802, 1101527, 1181109, 1247128, 1247851, 1320111, 1313047, 1371559, 1417832, 16918859, 18568495, 18573396, 18566132, 18565225, 18588861, 18587532, 18588482, 18586055, 46248114, 47204494, 25220922, 40947107, 43972849, 44116635, 54699343] 81


,pubchem,rt,inchi
0,5139,93.5,"InChI=1S/C3H8N2S/c1-2-6-3(4)5/h2H2,1H3,(H3,4,5)"
1,3505,687.8,InChI=1S/C19H25Cl2N3O3/c1-27-19(26)23-8-9-24(1...
2,2159,590.7,InChI=1S/C17H27N3O4S/c1-4-20-8-6-7-12(20)11-19...
3,1340,583.6,InChI=1S/C9H7NO2/c11-8-3-1-2-7-6(8)4-5-10-9(7)...
4,3344,579.0,InChI=1S/C15H20N2O2/c18-14-16-12-15(19-14)7-10...
...,...,...,...
80033,97733655,946.4,InChI=1S/C25H29N3O6S/c1-5-24(29)26-25-17(4)27-...
80034,98666786,653.1,InChI=1S/C17H24FN3O5S/c1-25-7-3-6-19-17(22)20-...
80035,98670835,648.2,InChI=1S/C17H25N3O5S/c1-13-4-3-5-16(8-13)26(22...
80036,98779314,783.9,InChI=1S/C21H25N3O4S/c1-15-7-9-19(10-8-15)29(2...


In [49]:
import pubchempy as pcp
for z in cid:
    c = pcp.Compound.from_cid(str(z))
    print(c)

Compound(298)
Compound(4477)
Compound(4022)
Compound(227516)
Compound(255871)
Compound(2777391)
Compound(2877960)
Compound(2919993)
Compound(2921419)
Compound(2948687)
Compound(2960737)
Compound(286826)
Compound(286876)
Compound(3123289)
Compound(309163)
Compound(3417144)
Compound(3463933)
Compound(3473868)
Compound(325419)
Compound(332278)
Compound(332429)
Compound(3565913)
Compound(3693695)
Compound(366651)
Compound(362112)
Compound(355940)
Compound(366721)
Compound(3854666)
Compound(3953303)
Compound(3979147)
Compound(4075240)
Compound(4176377)
Compound(4248743)
Compound(4427784)
Compound(4454455)
Compound(4532366)
Compound(4601040)
Compound(4647379)
Compound(28061)
Compound(5019999)
Compound(5064265)
Compound(5351231)
Compound(6002117)
Compound(6516747)
Compound(672387)
Compound(694700)
Compound(7205668)
Compound(711354)
Compound(7360729)
Compound(7518541)
Compound(7544267)
Compound(792698)
Compound(942434)
Compound(1040939)
Compound(1049537)
Compound(1071978)
Compound(1116802)
Com

In [50]:
leaving_behind=[]
for z in cid:
    c = pcp.Compound.from_cid(str(z))
    leaving_behind.append(c)

In [55]:
print(f["inchi"][cid[0]])
leaving_behind[0].to_dict(properties=['inchi'])

InChI=1S/C8H19NO/c1-7(9)5-4-6-8(2,3)10/h7,10H,4-6,9H2,1-3H3/t7-/m1/s1


{'inchi': 'InChI=1S/C11H12Cl2N2O5/c12-10(13)11(18)14-8(5-16)9(17)6-1-3-7(4-2-6)15(19)20/h1-4,8-10,16-17H,5H2,(H,14,18)'}